In [ ]:
!pip install --upgrade scenedetect[opencv] opencv-python numpy moviepy
!pip install scenedetect[opencv] opencv-python numpy moviepy

# Install ffmpeg and necessary codecs
!apt-get update
!apt install ffmpeg libavcodec-extra -y

import os
import cv2
import numpy as np
import subprocess
import tempfile

# ------------------ TUNABLE PARAMETERS ------------------ #
CLIP_DURATION = 15        # Desired clip length in seconds
MIN_CLIP_DURATION = 10    # If video is too short, extract 10s clips
FRAME_SAMPLE_RATE = 5     # Process every nth frame for motion analysis
#MOTION_THRESHOLD = 3.0    # Removed as it's not used in the std dev calculation
OVERLAP_THRESHOLD = 1     # Overlap allowed between clips (in seconds)
RESOLUTION = (1280, -2)   # Resize video for processing (width, height=-2 auto-scales)
SMOOTHNESS_THRESHOLD = 2.0  # Adjust for initial smoothness
STD_DEV_CHANGE_THRESHOLD = 0.15  # Adjust for rate of change
threshold = 180



def reencode_video(input_video_path, output_video_path, resolution):
    """Re-encodes the video for better processing if needed."""

    # 1. Probe the input video
    probe_cmd = [
        'ffprobe', '-v', 'error', '-select_streams', 'v:0',
        '-show_entries', 'stream=width,height,codec_name',
        '-of', 'csv=p=0', input_video_path
    ]
    probe_result = subprocess.run(probe_cmd, capture_output=True, text=True)
    if probe_result.returncode != 0:
        print(f"Error probing video: {probe_result.stderr}")
        return

    try:
        width, height, codec_name = probe_result.stdout.strip().split(',')
    except ValueError:
        print(f"Unexpected output from ffprobe: {probe_result.stdout}")
        return

    # 2. Build the re-encoding command
    cmd = [
        'ffmpeg', '-y', '-i', input_video_path,
        '-c:v', 'libx264', '-preset', 'medium',
        '-an',  # Disable audio
        '-vf', f'scale={resolution[0]}:{resolution[1]}',
        '-movflags', '+faststart', '-pix_fmt', 'yuv420p',
        output_video_path
    ]

    # 3. Check if re-encoding is needed (using probed info)
    if codec_name == 'h264' and int(width) <= resolution[0] and int(height) <= abs(resolution[1]):
        print("Input video already suitable. Copying instead of re-encoding.")
        cmd = ['ffmpeg', '-y', '-i', input_video_path, '-c:v', 'copy', '-an', output_video_path] # Disable audio

    # 4. Execute ffmpeg, handle moov atom error with remuxing
    try:
        result = subprocess.run(cmd, check=True, stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        if "moov atom not found" in e.stderr.decode('utf-8'):
            print("Remuxing video...")
            fixed_video_path = '/content/drone/fixed_video.mp4'
            if os.path.exists(fixed_video_path):
                os.remove(fixed_video_path)
                print(f"Overwriting existing {fixed_video_path}")
            remux_cmd = [
                'ffmpeg', '-y', '-i', input_video_path,
                '-c:v', 'copy',  # Copy video stream
                '-an',           # Disable audio in remuxing as well
                '-movflags', '+faststart',
                fixed_video_path
            ]
            subprocess.run(remux_cmd, check=True, stderr=subprocess.PIPE)
            input_video_path = fixed_video_path  # Update path
            cmd[cmd.index(input_video_path)] = fixed_video_path  # Update the command
            result = subprocess.run(cmd, check=True, stderr=subprocess.PIPE) # Retry
        else:
            print(f"Error during re-encoding: {e.stderr.decode('utf-8')}")
            return  # Exit if another error occurred during encoding

    if result.returncode == 0:
        print("Video re-encoded/copied successfully.")
    else:
        print(f"Re-encoding/copying failed with error code {result.returncode}.")


def remux_video(input_path, output_path):
    """Remuxes the video using ffmpeg to fix moov atom placement."""
    # Force overwrite by removing the existing file (if it exists)
    if os.path.exists(output_path):
        os.remove(output_path)
        print(f"Overwriting existing {output_path}")

    remux_cmd = [
        'ffmpeg', '-i', input_path, '-c', 'copy', '-movflags', 'faststart', output_path
    ]
    subprocess.run(remux_cmd, check=True, stderr=subprocess.PIPE)
    print(f"Video remuxed to: {output_path}")



# ------------------ PATHS ------------------ #
original_video_path = "/content/drone/DJI_0219.MP4"  # Store the original path
fixed_video_path = "/content/drone/fixed_video.mp4"
REENCODED_VIDEO_PATH = "/content/drone/reencoded_video.mp4"

# Force overwrite by removing the existing file (if it exists)
if os.path.exists(fixed_video_path):
    os.remove(fixed_video_path)
    print(f"Overwriting existing {fixed_video_path}")

# Force overwrite by removing the existing file (if it exists)
if os.path.exists(REENCODED_VIDEO_PATH):
    os.remove(REENCODED_VIDEO_PATH)
    print(f"Overwriting existing {REENCODED_VIDEO_PATH}")

# Remux the video using ffmpeg to fix moov atom placement
!ffmpeg -i {original_video_path} -c copy -movflags faststart {fixed_video_path}

# First re-encoding (or copying)
reencode_video(original_video_path, REENCODED_VIDEO_PATH, RESOLUTION)

# Update VIDEO_PATH for further processing
VIDEO_PATH = REENCODED_VIDEO_PATH

OUTPUT_FOLDER = "/content/drone/clips"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


# ------------------ FUNCTION: DETECT MOTION ------------------ #
def detect_smooth_sections(video_path, fps=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    motion_scores = []
    prev_gray = None

    for i in range(0, total_frames, FRAME_SAMPLE_RATE):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            motion_score = np.mean(np.abs(flow))
            motion_scores.append(motion_score)

        prev_gray = gray

    cap.release()

    # Calculate and display standard deviation and relative change per second
    motion_stds_per_second = []
    relative_changes = []

    for i in range(int(duration)):
        start_frame = int(i * fps)
        end_frame = int((i + 1) * fps)

        # Adjust frame range to consider sampling rate
        sampled_start_frame = start_frame // FRAME_SAMPLE_RATE
        sampled_end_frame = end_frame // FRAME_SAMPLE_RATE

        # Handle cases where the range is outside motion_scores
        sampled_end_frame = min(sampled_end_frame, len(motion_scores))

        if sampled_end_frame > sampled_start_frame:
            motion_std_second = np.std(motion_scores[sampled_start_frame:sampled_end_frame])
            motion_stds_per_second.append(motion_std_second)
        else:
            motion_stds_per_second.append(np.nan)  # Handle empty range

        # Calculate and display relative change
        if i > 0 and motion_stds_per_second[i - 1] != 0 and not np.isnan(motion_stds_per_second[i - 1]) and not np.isnan(motion_stds_per_second[i]):
            relative_change = ((motion_stds_per_second[i] - motion_stds_per_second[i - 1]) / motion_stds_per_second[i - 1]) * 100
            relative_changes.append(relative_change)
            print(f"Second {i + 1}: Motion Std Dev = {motion_std_second:.4f}, Relative Change = {relative_change:.2f}%")
        else:
            relative_changes.append(np.nan)
            print(f"Second {i + 1}: Motion Std Dev = {motion_std_second:.4f}, Relative Change = -")

    # Find smooth sections based on relative change and adaptive threshold
    smooth_sections = []
    clip_duration = CLIP_DURATION if duration >= CLIP_DURATION else MIN_CLIP_DURATION

    print(f"Checking for smooth segments with clip duration: {clip_duration}, threshold: {threshold}") # Debug: Print clip duration and threshold

    for i in range(len(relative_changes) - int(clip_duration) + 1):
        # Check if all relative changes within the clip duration are below the threshold
        is_smooth = all(abs(change) < threshold for change in relative_changes[i:i + int(clip_duration)] if not np.isnan(change))

        print(f"Checking segment {i}-{i + int(clip_duration)}: is_smooth = {is_smooth}") # Debug: Print segment and smoothness result

        if is_smooth:
            # If the current segment is smooth and not overlapping with existing clips, add it to smooth_sections
            is_overlapping = any(i <= s < i + int(clip_duration) for s in smooth_sections)
            if not is_overlapping:
                smooth_sections.append(i)
                avg_std_dev = np.mean(motion_stds_per_second[i:i + int(clip_duration)])
                print(f"Smooth clip found at {i}s with average std dev: {avg_std_dev:.4f}")
            else:
                print(f"Segment {i}-{i + int(clip_duration)} overlaps with existing clips.")  # Debug: Indicate overlap
        else:
            # Debug: Print the relative changes that exceeded the threshold for this segment
            exceeding_changes = [change for change in relative_changes[i:i + int(clip_duration)] if abs(change) >= threshold and not np.isnan(change)]
            if exceeding_changes:
                print(f"Segment {i}-{i + int(clip_duration)} rejected due to exceeding changes: {exceeding_changes}")

    print(f"Selected Smooth Clips (Start times in seconds): {smooth_sections}")  # Debug: Print selected clips

    # Extract non-overlapping clips (Modified to use smooth_sections directly)
    selected_clips = []
    for t in smooth_sections:
        if not any(abs(t - s) < (clip_duration - OVERLAP_THRESHOLD) for s in selected_clips):
            selected_clips.append(t)
    print(f"Final Selected Clips (after overlap removal): {selected_clips}")

    return selected_clips

# ------------------ FUNCTION: EXTRACT CLIPS ------------------ #
def extract_clips(video_path, output_folder, start_times, duration):
    for idx, start_time in enumerate(start_times):
        output_path = os.path.join(output_folder, f"clip_{idx + 1}.mp4")
        cmd = [
            'ffmpeg', '-y', '-ss', str(start_time), '-i', video_path,
            '-t', str(duration), '-c:v', 'libx264', '-preset', 'medium',
            '-an',  # No audio processing
            output_path
        ]
        subprocess.run(cmd, check=True, stderr=subprocess.PIPE)
        print(f"Extracted clip: {output_path} ({start_time:.2f}s to {start_time + duration:.2f}s)")

# ------------------ MAIN PROCESS ------------------ #
# Assuming VIDEO_PATH and OUTPUT_FOLDER are defined
selected_clips = detect_smooth_sections(VIDEO_PATH)

if selected_clips:
    extract_clips(VIDEO_PATH, OUTPUT_FOLDER, selected_clips, CLIP_DURATION)
else:
    print("No suitable smooth clips found.")